# INFO 323: Cloud Computing and Big Data
## CCI at Drexel University
## Counting MnM
### Yuan An

In [2]:
sc.version

'2.4.8'

# Spark Example: Counting MnM for the Cookie Monster

Let’s solve problem, but with a larger data set and using more of Spark’s distribution
functionality and DataFrame APIs. We will cover the APIs used in this program later.

Let’s write a Spark program that reads a file with over 100,000 entries (where each
row or line has a <state, mnm_color, count>) and computes and aggregates the
counts for each color and state. These aggregated counts tell us the colors of M&Ms
favored by students in each state.

### 1. Get the M&M data set filename from the command-line arguments

In [5]:
mnm_file = "gs://info323-ya45-bucket/notebooks/jupyter/mnm_dataset.csv"

### 2. Read the file into a Spark DataFrame using the CSV format by inferring the schema and specifying that the file contains a header, which provides column names for comma-separated fields.

In [6]:
mnm_df = (spark.read.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load(mnm_file))

### 3. We use the DataFrame high-level APIs. Note that we don't use RDDs at all. Because some of Spark's functions return the same object, we can chain function calls.
1. Select from the DataFrame the fields "State", "Color", and "Count"
2. Since we want to group each state and its M&M color count, we use groupBy()
3. Aggregate counts of all colors and groupBy() State and Color
4. orderBy() in descending order


In [8]:
from pyspark.sql.functions import count

count_mnm_df = (mnm_df
.select("State", "Color", "Count")
.groupBy("State", "Color")
.agg(count("Count").alias("Total"))
.orderBy("Total", ascending=False))

### 4. Show the resulting aggregations for all the states and colors; a total count of each color per state. Note show() is an action, which will trigger the above query to be executed.

In [8]:
count_mnm_df.show(n=60, truncate=False)
print("Total Rows = %d" % (count_mnm_df.count()))

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|WA   |Green |1779 |
|OR   |Orange|1743 |
|TX   |Green |1737 |
|TX   |Red   |1725 |
|CA   |Green |1723 |
|CO   |Yellow|1721 |
|CA   |Brown |1718 |
|CO   |Green |1713 |
|NV   |Orange|1712 |
|TX   |Yellow|1703 |
|NV   |Green |1698 |
|AZ   |Brown |1698 |
|WY   |Green |1695 |
|CO   |Blue  |1695 |
|NM   |Red   |1690 |
|AZ   |Orange|1689 |
|NM   |Yellow|1688 |
|NM   |Brown |1687 |
|UT   |Orange|1684 |
|NM   |Green |1682 |
|UT   |Red   |1680 |
|AZ   |Green |1676 |
|NV   |Yellow|1675 |
|NV   |Blue  |1673 |
|WA   |Red   |1671 |
|WY   |Red   |1670 |
|WA   |Brown |1669 |
|NM   |Orange|1665 |
|WY   |Blue  |1664 |
|WA   |Yellow|1663 |
|WA   |Orange|1658 |
|NV   |Brown |1657 |
|CA   |Orange|1657 |
|CO   |Brown |1656 |
|CA   |Red   |1656 |
|UT   |Blue  |1655 |
|AZ   |Yellow|1654 |
|TX   |Orange|1652 |
|AZ   |Red   |1648 |
|OR   |Blue  |1646 |
|UT   |Yellow|1645 |
|OR   |Red   |1645 |
|CO   |Orange|1642 |
|TX   |Brown 

### 5. While the above code aggregated and counted for all the states, what if we just want to see the data for a single state, e.g., CA?
1. Select from all rows in the DataFrame
2. Filter only CA state
3. groupBy() State and Color as we did above
4. Aggregate the counts for each color
5. orderBy() in descending order
6. Find the aggregate count for California by filtering

In [9]:
ca_count_mnm_df = (mnm_df
.select("State", "Color", "Count")
.where(mnm_df.State == "CA")
.groupBy("State", "Color")
.agg(count("Count").alias("Total"))
.orderBy("Total", ascending=False))

### Show the resulting aggregation for California. As above, show() is an action that will trigger the execution of the entire computation.

In [10]:
ca_count_mnm_df.show(n=10, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|CA   |Green |1723 |
|CA   |Brown |1718 |
|CA   |Orange|1657 |
|CA   |Red   |1656 |
|CA   |Blue  |1603 |
+-----+------+-----+

